Implementation of Naive_bayes algorithm

Import the necessary packages like numpy,pandas,matplotlib

In [1]:
import pandas as pd
import numpy as np

load the train and test datasets using pandas package

In [2]:
train=pd.read_csv("trainfsm.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test=pd.read_csv("test.csv")
test.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


check the shape of the both datasets

In [4]:
train.shape

(891, 12)

In [5]:
test.shape

(418, 11)

check for null values in the data

In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

fill the missing data with standard deviation values

In [9]:
female = train.loc[train['Sex'] == 'female', 'Age'].std()
male = train.loc[train['Sex'] == 'male', 'Age'].std()

# filling missing values in 'Age' column based on gender
train.loc[(train['Sex'] == 'female') & (train['Age'].isnull()), 'Age'] = female
train.loc[(train['Sex'] == 'male') & (train['Age'].isnull()), 'Age'] = male
test.loc[(test['Sex'] == 'female') & (test['Age'].isnull()), 'Age'] = female
test.loc[(test['Sex'] == 'male') & (test['Age'].isnull()), 'Age'] = male
test['Fare'] = test['Fare'].fillna(test['Fare'].std())

Now verify the null values

In [10]:
train.isnull().sum(),test.isnull().sum()

(PassengerId      0
 Survived         0
 Pclass           0
 Name             0
 Sex              0
 Age              0
 SibSp            0
 Parch            0
 Ticket           0
 Fare             0
 Cabin          687
 Embarked         2
 dtype: int64,
 PassengerId      0
 Pclass           0
 Name             0
 Sex              0
 Age              0
 SibSp            0
 Parch            0
 Ticket           0
 Fare             0
 Cabin          327
 Embarked         0
 dtype: int64)

combine the SibSp(siblings),Parch(parents) columns into family column

In [11]:
train['Family']=train['SibSp']+train['Parch']
del train['SibSp']
del train['Parch']

In [12]:
test['Family']=test['SibSp']+test['Parch']
del test['SibSp']
del test['Parch']


Drop the unwanted columns in the data

In [13]:
train=train.drop(['Ticket','Cabin','Name','PassengerId'],axis=1)
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,0,3,male,22.0,7.2500,S,1
1,1,1,female,38.0,71.2833,C,1
2,1,3,female,26.0,7.9250,S,0
3,1,1,female,35.0,53.1000,S,1
4,0,3,male,35.0,8.0500,S,0


In [14]:
test=test.drop(['Ticket','Cabin','Name','PassengerId'],axis=1)
test.head()

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,male,34.5,7.8292,Q,0
1,3,female,47.0,7.0000,S,1
2,2,male,62.0,9.6875,Q,0
3,3,male,27.0,8.6625,S,0
4,3,female,22.0,12.2875,S,2


convert the categorical values into integers using LabelEncoder

In [15]:
class LabelEncoder:
    def __init__(self):
        self.mapping = {}
    
    def fit(self, y):
        unique_labels = set(y)
        for i, label in enumerate(unique_labels):
            self.mapping[label] = i
    
    def transform(self, y):
        encoded_labels = []
        for label in y:
            encoded_labels.append(self.mapping[label])
        return encoded_labels
    
    def fit_transform(self, y):
        self.fit(y)
        return self.transform(y)

In [16]:
label_sex = LabelEncoder()
label_embark = LabelEncoder()
train['Sex'] = label_sex.fit_transform(train['Sex'])
train['Embarked'] = label_embark.fit_transform(train['Embarked'])
test['Sex'] = label_sex.fit_transform(test['Sex'])
test['Embarked'] = label_embark.fit_transform(test['Embarked'])

In [17]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,0,3,0,22.0,7.2500,1,1
1,1,1,1,38.0,71.2833,0,1
2,1,3,1,26.0,7.9250,1,0
3,1,1,1,35.0,53.1000,1,1
4,0,3,0,35.0,8.0500,1,0


In [18]:
test.head()

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,0,34.5,7.8292,0,0
1,3,1,47.0,7.0000,1,1
2,2,0,62.0,9.6875,0,0
3,3,0,27.0,8.6625,1,0
4,3,1,22.0,12.2875,1,2


now seperate the Survived Column(target variable) from train dataset

In [19]:
y = train['Survived']
del train['Survived']

In [20]:
y.head()


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [21]:
def train_test_split(data, target, test_size=0.2, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)
    
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    
    split_index = int(len(data) * (1 - test_size))
    train_indices = indices[:split_index]
    test_indices = indices[split_index:]
    
    train_data = data.iloc[train_indices]
    test_data = data.iloc[test_indices]
    train_target = target.iloc[train_indices]
    test_target = target.iloc[test_indices]
    
    return train_data, test_data, train_target, test_target


In [22]:
train_data, test_data, train_target, test_target = train_test_split(train, y, test_size=0.2, random_seed=42)

create the naive_bayes model

In [23]:
class NaiveBayes:
    def __init__(self):
        self.prior = {}
        self.conditional = {}

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.num_classes = len(self.classes)

        for c in self.classes:
            self.prior[c] = np.mean(y == c)

            X_c = X[y == c]
            self.conditional[c] = {}
            for feature in X.columns:
                self.conditional[c][feature] = {
                    'mean': np.mean(X_c[feature]),
                    'std': np.std(X_c[feature])
                }

    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            posterior = {}
            for c in self.classes:
                likelihood = 1
                for feature in X.columns:
                    mean = self.conditional[c][feature]['mean']
                    std = self.conditional[c][feature]['std']
                    x = row[feature]
                    likelihood *= self.calculate_likelihood(x, mean, std)
                posterior[c] = likelihood * self.prior[c]
            prediction = max(posterior, key=posterior.get)
            predictions.append(prediction)
        return predictions

    def calculate_likelihood(self, x, mean, std):
        exponent = np.exp(-((x - mean) ** 2) / (2 * (std ** 2)))
        likelihood = (1 / (np.sqrt(2 * np.pi) * std)) * exponent
        return likelihood


In [24]:
naive_bayes = NaiveBayes()
naive_bayes.fit(train, y) # fitting values
predictions = naive_bayes.predict(test) #prediction of test date

import the validation dataset

In [25]:
valid = pd.read_csv("gender_submission.csv" , usecols=['Survived']) # avoid the PassengerId column
valid.head()

,Survived
0,0
1,1
2,0
3,0
4,1


convert the valid data into list data to calculate accuracy_score

In [26]:
p_list = valid.values.tolist()

determine the accuracy_score function

In [28]:
def accuracy_score(y_true, y_pred):
    correct = 0
    total = len(y_true)

    for i in range(total):
        if y_true[i] == y_pred[i]:
            correct += 1

    accuracy = correct / total
    return accuracy

calculate the accuracy value

In [31]:
accuracy = accuracy_score(p_list, predictions)*100
print("Accuracy:", accuracy)


Accuracy: 91.86602870813397
